In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2

# ---------------- 1️⃣ Data Preprocessing ----------------
# Define data augmentation techniques
datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=30,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    fill_mode='nearest',
    validation_split=0.2  # 80% Train, 20% Validation
)

# Load dataset from directory
train_data = datagen.flow_from_directory(
    "nail_dataset/train",
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    "nail_dataset/train",
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# ---------------- 2️⃣ Use MobileNetV2 (Pretrained Model) ----------------
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Define model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 classes: Healthy, Onychomycosis, Psoriasis
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ---------------- 3️⃣ Train Model with Early Stopping ----------------
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stop]
)

# ---------------- 4️⃣ Save the Trained Model ----------------
model.save("nail_disease_model.h5")
print("✅ Model saved successfully!")

# ---------------- 5️⃣ Evaluate on Test Data ----------------
def load_test_data(test_folder):
    images = []
    labels = []
    class_names = ["healthy", "onychomycosis", "psoriasis"]
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(test_folder, class_name)
        for img_name in os.listdir(class_folder):
            if img_name.endswith(".jpg") or img_name.endswith(".png"):
                img_path = os.path.join(class_folder, img_name)
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
                img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
                images.append(img_array)
                labels.append(idx)
    
    return np.array(images), np.array(labels)

X_test, y_test = load_test_data("nail_dataset/test")
y_pred = np.argmax(model.predict(X_test), axis=1)

from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, y_pred, target_names=["Healthy", "Onychomycosis", "Psoriasis"]))
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# ---------------- 6️⃣ Predict on a New Image ----------------
def predict_nail_condition(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array, verbose=0)
    class_names = ["Healthy", "Onychomycosis", "Psoriasis"]
    return class_names[np.argmax(prediction)]

# Example usage
test_image_path = "nail_dataset/test/onychomycosis/7.jpg"
predicted_label = predict_nail_condition(test_image_path)
print(f"🩺 Predicted Condition: {predicted_label}")


Found 928 images belonging to 3 classes.
Found 231 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step


c:\Users\ronit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3610 - loss: 1.6642 

c:\Users\ronit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.3630 - loss: 1.6575 - val_accuracy: 0.6277 - val_loss: 0.8927
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 305ms/step - accuracy: 0.5801 - loss: 0.9943 - val_accuracy: 0.6753 - val_loss: 0.7619
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 304ms/step - accuracy: 0.6963 - loss: 0.7476 - val_accuracy: 0.6840 - val_loss: 0.7652
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 302ms/step - accuracy: 0.7217 - loss: 0.6904 - val_accuracy: 0.6883 - val_loss: 0.6790
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 323ms/step - accuracy: 0.7920 - loss: 0.5442 - val_accuracy: 0.7186 - val_loss: 0.6397
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 331ms/step - accuracy: 0.7551 - loss: 0.5568 - val_accuracy: 0.7316 - val_loss: 0.6466
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 348ms/step - accuracy: 0.8143 - loss: 0.4812 - val_accuracy: 0.7316 - val_loss: 0.6414
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 15s 513ms/step - accuracy: 0.8313 - loss: 0.4063 - val_accuracy: 0.7229 - val

✅ Model saved successfully!
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step
               precision    recall  f1-score   support

      Healthy       0.96      0.74      0.83        61
Onychomycosis       0.72      0.95      0.82       145
    Psoriasis       0.84      0.53      0.65        91

     accuracy                           0.78       297
    macro avg       0.84      0.74      0.77       297
 weighted avg       0.80      0.78      0.77       297

✅ Test Accuracy: 0.7778
🩺 Predicted Condition: Onychomycosis


In [9]:
# Example usage
test_image_path = "nail_dataset/test/healthy/56.jpg"
predicted_label = predict_nail_condition(test_image_path)
print(f"🩺 Predicted Condition: {predicted_label}")


🩺 Predicted Condition: Healthy
